# Task Master Prompt Chaining Analysis

This notebook analyzes how prompts are generated, chained, and processed in the Task Master system, focusing on the AI-driven task generation functionality.

## 1. Overview of Task Master's AI Integration

Task Master uses a layered architecture for AI integration that powers several key commands:

1. `parse-prd` - Transforms text requirements into structured tasks
2. `analyze-complexity` - Assesses task complexity and recommends subtask counts
3. `expand-task` - Breaks down tasks into detailed subtasks
4. `update-task` - Updates tasks based on new context or requirements
5. `add-task` - Creates new tasks with AI assistance

Each of these commands relies on prompt generation and AI response processing through a unified service layer.

## 2. The AI Service Layer

The central integration point for all AI operations is `ai-services-unified.js`, which provides three main functions:

- `generateTextService` - For unstructured text responses
- `generateObjectService` - For structured JSON outputs with Zod validation
- `streamTextService` - For streaming responses

Let's examine how this layer works:

In [ ]:
// From scripts/modules/ai-services-unified.js

// Provider function mapping
const PROVIDER_FUNCTIONS = {
    anthropic: {
        generateText: anthropic.generateAnthropicText,
        streamText: anthropic.streamAnthropicText,
        generateObject: anthropic.generateAnthropicObject
    },
    perplexity: {
        generateText: perplexity.generatePerplexityText,
        streamText: perplexity.streamPerplexityText,
        generateObject: perplexity.generatePerplexityObject
    },
    // Additional providers...
};

// Main service functions
async function generateTextService(params) {
    return _unifiedServiceRunner('generateText', params);
}

async function generateObjectService(params) {
    const defaults = {
        objectName: 'generated_object',
        maxRetries: 3
    };
    const combinedParams = { ...defaults, ...params };
    return _unifiedServiceRunner('generateObject', combinedParams);
}

// Core service runner with role-based fallbacks
async function _unifiedServiceRunner(serviceType, params) {
    const {
        role: initialRole,
        session,
        projectRoot,
        systemPrompt,
        prompt,
        schema,
        objectName,
        ...restApiParams
    } = params;
    
    // Determine fallback sequence based on initial role
    let sequence;
    if (initialRole === 'main') {
        sequence = ['main', 'fallback', 'research'];
    } else if (initialRole === 'research') {
        sequence = ['research', 'fallback', 'main'];
    } else if (initialRole === 'fallback') {
        sequence = ['fallback', 'main', 'research'];
    } else {
        sequence = ['main', 'fallback', 'research'];
    }

    // Try each role in sequence until success
    for (const currentRole of sequence) {
        // Get provider and model config
        // Create messages array
        // Attempt API call with retries
        // Return on success
    }
    
    // If loop completes, all roles failed
    throw new Error('All roles in the sequence failed.');
}

### Key aspects of the AI service layer:

1. **Provider Abstraction**: Maps different LLM providers (Anthropic, Perplexity, OpenAI, etc.) to consistent function signatures
2. **Role-Based Fallbacks**: Attempts calls in a sequence based on role (main → fallback → research)
3. **Structured Response Validation**: For `generateObjectService`, uses Zod schemas to validate responses
4. **Resilience**: Includes retry logic with exponential backoff for transient errors

## 3. PRD Parsing: From Text to Tasks

The `parse-prd` command is the starting point of the task generation workflow. It takes a text requirements document and transforms it into structured tasks.

In [ ]:
// From scripts/modules/task-manager/parse-prd.js

// Zod schema for validation
const prdSingleTaskSchema = z.object({
    id: z.number().int().positive(),
    title: z.string().min(1),
    description: z.string().min(1),
    details: z.string().optional().default(''),
    testStrategy: z.string().optional().default(''),
    priority: z.enum(['high', 'medium', 'low']).default('medium'),
    dependencies: z.array(z.number().int().positive()).optional().default([]),
    status: z.string().optional().default('pending')
});

const prdResponseSchema = z.object({
    tasks: z.array(prdSingleTaskSchema),
    metadata: z.object({
        projectName: z.string(),
        totalTasks: z.number(),
        sourceFile: z.string(),
        generatedAt: z.string()
    })
});

// System prompt for PRD parsing
const systemPrompt = `You are an AI assistant specialized in analyzing Product Requirements Documents (PRDs) and generating a structured, logically ordered, dependency-aware and sequenced list of development tasks in JSON format.
Analyze the provided PRD content and generate approximately ${numTasks} top-level development tasks. If the complexity or the level of detail of the PRD is high, generate more tasks relative to the complexity of the PRD
Each task should represent a logical unit of work needed to implement the requirements and focus on the most direct and effective way to implement the requirements without unnecessary complexity or overengineering. Include pseudo-code, implementation details, and test strategy for each task. Find the most up to date information to implement each task.
Assign sequential IDs starting from ${nextId}. Infer title, description, details, and test strategy for each task based *only* on the PRD content.
Set status to 'pending', dependencies to an empty array [], and priority to 'medium' initially for all tasks.
Respond ONLY with a valid JSON object containing a single key "tasks", where the value is an array of task objects adhering to the provided Zod schema. Do not include any explanation or markdown formatting.

Each task should follow this JSON structure:
{
    "id": number,
    "title": string,
    "description": string,
    "status": "pending",
    "dependencies": number[] (IDs of tasks this depends on),
    "priority": "high" | "medium" | "low",
    "details": string (implementation details),
    "testStrategy": string (validation approach)
}

Guidelines:
1. Unless complexity warrants otherwise, create exactly ${numTasks} tasks, numbered sequentially starting from ${nextId}
2. Each task should be atomic and focused on a single responsibility following the most up to date best practices and standards
3. Order tasks logically - consider dependencies and implementation sequence
4. Early tasks should focus on setup, core functionality first, then advanced features
5. Include clear validation/testing approach for each task
6. Set appropriate dependency IDs (a task can only depend on tasks with lower IDs, potentially including existing tasks with IDs less than ${nextId} if applicable)
7. Assign priority (high/medium/low) based on criticality and dependency order
8. Include detailed implementation guidance in the "details" field
9. If the PRD contains specific requirements for libraries, database schemas, frameworks, tech stacks, or any other implementation details, STRICTLY ADHERE to these requirements in your task breakdown and do not discard them under any circumstance
10. Focus on filling in any gaps left by the PRD or areas that aren't fully specified, while preserving all explicit requirements
11. Always aim to provide the most direct path to implementation, avoiding over-engineering or roundabout approaches`;

// User prompt with PRD content
const userPrompt = `Here's the Product Requirements Document (PRD) to break down into approximately ${numTasks} tasks, starting IDs from ${nextId}:\n\n${prdContent}\n\n

Return your response in this format:
{
    "tasks": [
        {
            "id": 1,
            "title": "Setup Project Repository",
            "description": "...",
            ...
        },
        ...
    ],
    "metadata": {
        "projectName": "PRD Implementation",
        "totalTasks": ${numTasks},
        "sourceFile": "${prdPath}",
        "generatedAt": "YYYY-MM-DD"
    }
}`;

// Call the unified AI service
const generatedData = await generateObjectService({
    role: 'main',
    session: session,
    projectRoot: projectRoot,
    schema: prdResponseSchema,
    objectName: 'tasks_data',
    systemPrompt: systemPrompt,
    prompt: userPrompt,
    reportProgress
});

### Key aspects of PRD parsing:

1. **Structured Output**: Uses `generateObjectService` with a Zod schema to ensure valid task structure
2. **Detailed Instructions**: The system prompt provides comprehensive guidelines for task generation
3. **Dependency Management**: Post-processes the AI response to remap dependencies and ensure proper task sequencing
4. **Task Normalization**: Sets default values and ensures consistent task structure

After generating tasks, the system writes them to `tasks.json` and generates individual task files.

## 4. Complexity Analysis: Sizing and Prioritizing Tasks

The `analyze-complexity` command evaluates tasks and determines how complex they are, which affects how they're broken down later.

In [ ]:
// From scripts/modules/task-manager/analyze-task-complexity.js

/**
 * Generates the prompt for complexity analysis.
 */
function generateInternalComplexityAnalysisPrompt(tasksData) {
    const tasksString = JSON.stringify(tasksData.tasks, null, 2);
    return `Analyze the following tasks to determine their complexity (1-10 scale) and recommend the number of subtasks for expansion. Provide a brief reasoning and an initial expansion prompt for each.

Tasks:
${tasksString}

Respond ONLY with a valid JSON array matching the schema:
[
  {
    "taskId": <number>,
    "taskTitle": "<string>",
    "complexityScore": <number 1-10>,
    "recommendedSubtasks": <number>,
    "expansionPrompt": "<string>",
    "reasoning": "<string>"
  },
  ...
]

Do not include any explanatory text, markdown formatting, or code block markers before or after the JSON array.`;
}

// System prompt for complexity analysis
const systemPrompt =
    'You are an expert software architect and project manager analyzing task complexity. Respond only with the requested valid JSON array.';

// AI service call
const role = useResearch ? 'research' : 'main';
fullResponse = await generateTextService({
    prompt,
    systemPrompt,
    role,
    session,
    projectRoot
});

// Then parse and process the response into a complexity report
// This includes JSON parsing with fallbacks for code blocks, error handling, etc.

### Key aspects of complexity analysis:

1. **Scoring System**: Tasks are scored on a 1-10 complexity scale
2. **Subtask Recommendations**: Each task gets a recommended number of subtasks based on complexity
3. **Custom Expansion Prompts**: Generates task-specific prompts for later expansion
4. **Reasoning**: Includes reasoning for each complexity assessment
5. **Report Generation**: Creates a report at `scripts/task-complexity-report.json` for later use

This is a critical step in the chain as it influences how tasks are expanded and broken down in the next phase.

## 5. Task Expansion: Breaking Down Tasks into Subtasks

The `expand-task` command uses the task details and optional complexity analysis to break tasks down into actionable subtasks.

In [ ]:
// From scripts/modules/task-manager/expand-task.js

/**
 * Generates the system prompt for the main AI role.
 */
function generateMainSystemPrompt(subtaskCount) {
    return `You are an AI assistant helping with task breakdown for software development.
You need to break down a high-level task into ${subtaskCount} specific subtasks that can be implemented one by one.

Subtasks should:
1. Be specific and actionable implementation steps
2. Follow a logical sequence
3. Each handle a distinct part of the parent task
4. Include clear guidance on implementation approach
5. Have appropriate dependency chains between subtasks (using the new sequential IDs)
6. Collectively cover all aspects of the parent task

For each subtask, provide:
- id: Sequential integer starting from the provided nextSubtaskId
- title: Clear, specific title
- description: Detailed description
- dependencies: Array of prerequisite subtask IDs (use the new sequential IDs)
- details: Implementation details
- testStrategy: Optional testing approach


Respond ONLY with a valid JSON object containing a single key "subtasks" whose value is an array matching the structure described. Do not include any explanatory text, markdown formatting, or code block markers.`;
}

/**
 * Generates the user prompt for the main AI role.
 */
function generateMainUserPrompt(
    task,
    subtaskCount,
    additionalContext,
    nextSubtaskId
) {
    const contextPrompt = additionalContext
        ? `\n\nAdditional context: ${additionalContext}`
        : '';
    const schemaDescription = `
{
  "subtasks": [
    {
      "id": ${nextSubtaskId}, // First subtask ID
      "title": "Specific subtask title",
      "description": "Detailed description",
      "dependencies": [], // e.g., [${nextSubtaskId + 1}] if it depends on the next
      "details": "Implementation guidance",
      "testStrategy": "Optional testing approach"
    },
    // ... (repeat for a total of ${subtaskCount} subtasks with sequential IDs)
  ]
}`;

    return `Break down this task into exactly ${subtaskCount} specific subtasks:

Task ID: ${task.id}
Title: ${task.title}
Description: ${task.description}
Current details: ${task.details || 'None'}
${contextPrompt}

Return ONLY the JSON object containing the "subtasks" array, matching this structure:
${schemaDescription}`;
}

// Complexity report integration
try {
    if (fs.existsSync(complexityReportPath)) {
        const complexityReport = readJSON(complexityReportPath);
        taskAnalysis = complexityReport?.complexityAnalysis?.find(
            (a) => a.taskId === task.id
        );
        if (taskAnalysis) {
            logger.info(
                `Found complexity analysis for task ${task.id}: Score ${taskAnalysis.complexityScore}`
            );
            if (taskAnalysis.reasoning) {
                complexityReasoningContext = `\nComplexity Analysis Reasoning: ${taskAnalysis.reasoning}`;
            }
        }
    }
} catch (reportError) {
    logger.warn(
        `Could not read or parse complexity report: ${reportError.message}. Proceeding without it.`
    );
}

// Determine prompt content
if (taskAnalysis?.expansionPrompt) {
    // Use prompt from complexity report
    promptContent = taskAnalysis.expansionPrompt;
    // Append additional context and reasoning
    promptContent += `\n\n${additionalContext}`.trim();
    promptContent += `${complexityReasoningContext}`.trim();

    // Use simplified system prompt for report prompts
    systemPrompt = `You are an AI assistant helping with task breakdown. Generate exactly ${finalSubtaskCount} subtasks based on the provided prompt and context. Respond ONLY with a valid JSON object containing a single key "subtasks" whose value is an array of the generated subtask objects. Each subtask object in the array must have keys: "id", "title", "description", "dependencies", "details", "status". Ensure the 'id' starts from ${nextSubtaskId} and is sequential. Ensure 'dependencies' only reference valid prior subtask IDs generated in this response (starting from ${nextSubtaskId}). Ensure 'status' is 'pending'. Do not include any other text or explanation.`;
} else {
    // Use standard prompt generation
    const combinedAdditionalContext =
        `${additionalContext}${complexityReasoningContext}`.trim();
    if (useResearch) {
        promptContent = generateResearchUserPrompt(
            task,
            finalSubtaskCount,
            combinedAdditionalContext,
            nextSubtaskId
        );
        systemPrompt = `You are an AI assistant that responds ONLY with valid JSON objects as requested. The object should contain a 'subtasks' array.`;
    } else {
        promptContent = generateMainUserPrompt(
            task,
            finalSubtaskCount,
            combinedAdditionalContext,
            nextSubtaskId
        );
        systemPrompt = generateMainSystemPrompt(finalSubtaskCount);
    }
}

// Call generateTextService with the determined prompts
responseText = await generateTextService({
    prompt: promptContent,
    systemPrompt: systemPrompt,
    role,
    session,
    projectRoot
});

### Key aspects of task expansion:

1. **Complexity Report Integration**: Uses complexity analysis if available to determine subtask count and custom prompts
2. **Two Prompt Generation Modes**:
   - Standard mode with detailed structure guidance
   - Research mode for more exploratory expansion
3. **Dynamic Prompt Selection**: Chooses between complexity report prompts and standard prompts based on availability
4. **Subtask Validation**: Uses Zod schemas to ensure subtasks follow the correct structure
5. **Sequential ID Management**: Ensures subtasks have proper sequential IDs
6. **Dependency Chain Support**: Facilitates dependency relationships between subtasks

## 6. Task Updates: Evolving Tasks with New Context

The `update-task` command modifies existing tasks based on new requirements or context while preserving completed work.

In [ ]:
// From scripts/modules/task-manager/update-task-by-id.js

// System prompt for task updates
const systemPrompt = `You are an AI assistant helping to update a software development task based on new context.
You will be given a task and a prompt describing changes or new implementation details.
Your job is to update the task to reflect these changes, while preserving its basic structure.

Guidelines:
1. VERY IMPORTANT: NEVER change the title of the task - keep it exactly as is
2. Maintain the same ID, status, and dependencies unless specifically mentioned in the prompt
3. Update the description, details, and test strategy to reflect the new information
4. Do not change anything unnecessarily - just adapt what needs to change based on the prompt
5. Return a complete valid JSON object representing the updated task
6. VERY IMPORTANT: Preserve all subtasks marked as "done" or "completed" - do not modify their content
7. For tasks with completed subtasks, build upon what has already been done rather than rewriting everything
8. If an existing completed subtask needs to be changed/undone based on the new context, DO NOT modify it directly
9. Instead, add a new subtask that clearly indicates what needs to be changed or replaced
10. Use the existence of completed subtasks as an opportunity to make new subtasks more specific and targeted
11. Ensure any new subtasks have unique IDs that don't conflict with existing ones

The changes described in the prompt should be thoughtfully applied to make the task more accurate and actionable.`;

const taskDataString = JSON.stringify(taskToUpdate, null, 2);
const userPrompt = `Here is the task to update:\n${taskDataString}\n\nPlease update this task based on the following new context:\n${prompt}\n\nIMPORTANT: In the task JSON above, any subtasks with "status": "done" or "status": "completed" should be preserved exactly as is. Build your changes around these completed items.\n\nReturn only the updated task as a valid JSON object.`;

// Call AI service
responseText = await generateTextService({
    prompt: userPrompt,
    systemPrompt: systemPrompt,
    role,
    session,
    projectRoot
});

// Parse response and perform various validations
updatedTask = parseUpdatedTaskFromText(
    responseText,
    taskId,
    logFn,
    isMCP
);

// Preserve completed subtasks
if (taskToUpdate.subtasks?.length > 0) {
    if (!updatedTask.subtasks) {
        updatedTask.subtasks = taskToUpdate.subtasks;
    } else {
        const completedOriginal = taskToUpdate.subtasks.filter(
            (st) => st.status === 'done' || st.status === 'completed'
        );
        completedOriginal.forEach((compSub) => {
            const updatedSub = updatedTask.subtasks.find(
                (st) => st.id === compSub.id
            );
            if (
                !updatedSub ||
                JSON.stringify(updatedSub) !== JSON.stringify(compSub)
            ) {
                // Remove potentially modified version
                updatedTask.subtasks = updatedTask.subtasks.filter(
                    (st) => st.id !== compSub.id
                );
                // Add back original
                updatedTask.subtasks.push(compSub);
            }
        });
    }
}

### Key aspects of task updates:

1. **Work Preservation**: Special handling to maintain completed subtasks and prevent modifications
2. **Structural Integrity**: Guidelines to preserve core task structure while updating content
3. **Task ID Protection**: Ensures task IDs remain consistent
4. **Dependency Maintenance**: Preserved dependencies unless explicitly changed
5. **Conflict Resolution**: Logic for handling conflicts when completed subtasks need revisions

## 7. Adding Tasks: Creating New Tasks with AI

The `add-task` command creates new tasks with appropriate integration into the existing task structure.

In [ ]:
// From mcp-server/src/core/direct-functions/add-task.js

// Zod Schema for task validation
const AiTaskDataSchema = z.object({
    title: z.string().min(1),
    description: z.string().min(1),
    details: z.string(),
    testStrategy: z.string().optional(),
    priority: z.enum(['high', 'medium', 'low']).default('medium'),
    dependencies: z.array(z.number()).optional().default([])
});

// System prompt for task creation
const systemPrompt = 'You are an AI assistant helping to create well-structured development tasks. Respond only with the requested JSON object.';

// Assemble context information for the AI
const existingTasksInfo = getExistingTasksInfo(tasksData.tasks);
const dependenciesInfo = initialDependencies.length > 0 
    ? `\nThis task should depend on task IDs: ${initialDependencies.join(', ')}`
    : '';
const priorityInfo = initialPriority 
    ? `\nThis task should have priority: ${initialPriority}`
    : '';

// User prompt with context
const userPrompt = `Create a new development task based on this description: ${prompt}\n\n${existingTasksInfo}${dependenciesInfo}${priorityInfo}\n\nReturn ONLY a valid JSON object with these fields:\n- title: A clear, specific title\n- description: Detailed description\n- details: Implementation guidance\n- testStrategy: Validation approach\n- priority: "high", "medium", or "low"\n- dependencies: Array of task IDs this depends on`;

// Generate task data
const taskData = await generateObjectService({
    role: useResearch ? 'research' : 'main',
    schema: AiTaskDataSchema,
    objectName: 'task_data',
    systemPrompt,
    prompt: userPrompt,
    session,
    projectRoot
});

// Verify dependencies and add the task
validateDependencies(taskData.dependencies, tasksData.tasks);
const newTask = {
    id: nextId,
    title: taskData.title,
    description: taskData.description,
    status: 'pending',
    dependencies: taskData.dependencies,
    priority: taskData.priority,
    details: taskData.details,
    testStrategy: taskData.testStrategy || '',
    subtasks: []
};

tasksData.tasks.push(newTask);

### Key aspects of adding tasks:

1. **Context-Aware Generation**: Provides information about existing tasks to the AI
2. **Dependency Management**: Validates that dependencies refer to existing tasks
3. **Structured Output**: Uses `generateObjectService` with a schema to ensure proper structure
4. **ID Allocation**: Assigns the next available ID to the new task
5. **Priority Assignment**: Sets priority based on user input or AI recommendation

## 8. Command Chaining: How Commands Work Together

Task Master's commands typically form chains where the output of one command serves as input to another. The most common chains are:

### Initial Project Setup Chain

```
parse-prd → analyze-complexity → expand-task
```

1. `parse-prd` generates high-level tasks from requirements
2. `analyze-complexity` assesses each task's complexity and recommends subtask counts
3. `expand-task` breaks down tasks into actionable subtasks

### Ongoing Development Chain

```
add-task → update-task → expand-task
```

1. `add-task` creates new tasks as requirements evolve
2. `update-task` modifies tasks based on new context or information
3. `expand-task` breaks down updated tasks into detailed subtasks

### Integration Between Commands

Commands integrate through:

1. **File-based Integration**: `tasks.json` serves as the central data store
2. **Complexity Report**: `task-complexity-report.json` links analysis to expansion
3. **Task ID References**: IDs and dependencies maintain relationships
4. **Shared Service Layer**: The unified AI service provides consistent interaction

This system allows for an iterative, flexible workflow that adapts to changing requirements while maintaining consistency.

## 9. Conclusion: Key Design Patterns in Prompt Generation

From our analysis, we can identify several key patterns in Task Master's prompt engineering approach:

1. **Structured Output Templates**: JSON schemas and examples guide the AI to produce consistent outputs
2. **Context Preservation**: Tasks retain completed work while evolving with new requirements
3. **Cascading Complexity**: Analysis results guide task expansion, creating a more adaptive workflow
4. **Role-Based Fallbacks**: Different AI models serve different purposes with automatic fallback mechanisms
5. **Task Dependency Management**: Complex dependency relationships are maintained through the entire workflow
6. **Robust Parsing Logic**: Sophisticated parsing handles various AI response formats and potential errors
7. **Intermediate Artifacts**: Complexity reports and other artifacts connect different commands
8. **Validation Pipelines**: Zod schemas ensure consistent, valid outputs throughout the system

These patterns together form a robust, flexible system for transforming high-level requirements into structured, actionable development tasks.